In [1]:
from T13BFLSimpCore import *

In [2]:
dat = 'trips_simpler.csv'
company = [2,3,4,5]
location = [mk for mk in range(9)]

blockchain = Blockchain()

list index out of range|Brand new blockchain, creating genesis block
BLK: C:0x2053a#  0 <-- PH=0    :PF=0    , RL={}   , #TRXS=0
Blockchain created with ID: 0x2053a29bf88 2221473906568


In [3]:
print("1 - DISTRIBUTION =================================")
import pickle
try:
    locdump = "./models/clis" + str(company) + str(location) + ".p"
    clients = pickle.load(open(locdump, "rb"))
    print("Auto-load from cache")
except:
    print("Generating dist")
    db = Distributor(dat, company, location, settings)
    db.divBoth(settings)
    clients = db.extractClients()
    import pickle
    pickle.dump(clients, open(locdump, "wb"))

1 - DISTRIBUTION =================================
Auto-load from cache


In [4]:
print("2 - AUTHORIZATION =================================")
print("Using blockchain #" + blockchain.bc_id)
for c in clients:
    c.model = CNNLSTM(settings['frame_in'],settings['frame_out'], epochs=c.epoch)
    blockchain.authorizeClient(c)
    c.assignBlockchain(blockchain)
    print(c)

2 - AUTHORIZATION =================================
Using blockchain #0x2053a29bf88
CLI:2/0--2528,    1001
CLI:2/1-31361,    2464
CLI:2/2--3876,     357
CLI:2/3--2963,      89
CLI:2/4-10659,    5282
CLI:2/5-27101,    1224
CLI:2/6-62759,    2259
CLI:2/7-43950,    1603
CLI:2/8-59722,    2461
MIN:3/0--1471,   61603
CLI:3/1-28316,   45255
CLI:3/2--1305,    8442
CLI:3/3--5973,    5378
MIN:3/4-47149,  131480
CLI:3/5--3308,   19540
CLI:3/6--4737,   45318
CLI:3/7-10615,   27896
CLI:3/8--8878,   38346
CLI:4/0-65882,     582
CLI:4/1--5911,    3256
CLI:4/2-39966,     158
CLI:4/3--9697,      47
CLI:4/4--3538,   23207
CLI:4/5--8112,      58
CLI:4/6-64731,     766
CLI:4/7-74989,     135
CLI:4/8--5852,     532
CLI:5/0-21492,   12036
CLI:5/1-35076,   21358
CLI:5/2-28710,    3898
CLI:5/3-32441,    1306
MIN:5/4--5996,   43917
CLI:5/5-86801,    5967
CLI:5/6-10392,   16614
CLI:5/7-73968,    7205
CLI:5/8-43609,   12983


In [5]:
def train(ep=None):
    print("\n3 - TRAINING =================================")
    for c in clients:
        c.model.setData(c.inp_train,c.out_train)
        c.train(epoch=ep)
        c.submitModelAsTransaction()

def mine():
    print("\n4 - MINING =================================")
    for c in clients:
        if type(c) == Miner:
            c.mine()

def consensus():
    print("\n5 - CONSENSUS + INCENTIVIZATION ============")
    blockchain.consensus()

def aggregate():
    print("\n6 - AGGREGATION ============================", end=' ')
    for c in clients:
        mods = []
        for tx in c.blockchain.lastBlock.transactions:
            if tx.type == "MOD":
                mods.append(tx.load)
        if len(mods) == 0:
            print("No aggregation.\n")
            return
        nmod = c.aggregate(mods)
        c.replaceWeights(nmod)
    print("done.\n")

def iterate():
    train(37)
    mine()
    consensus()
    aggregate()

In [6]:
# FL iterations
iters = 3
for i in range(iters):
    print("> ITERATION #" + str(i+1))
    iterate()

> ITERATION #1

3 - TRAINING =================================
#2/0:6.64s; CLI:2/0 MOD-->CHIDX#  1
#2/1:6.41s; CLI:2/1 MOD-->CHIDX#  1
#2/2:6.43s; CLI:2/2 MOD-->CHIDX#  1
#2/3:7.30s; CLI:2/3 MOD-->CHIDX#  1
#2/4:6.59s; CLI:2/4 MOD-->CHIDX#  1
#2/5:6.34s; CLI:2/5 MOD-->CHIDX#  1
#2/6:6.36s; CLI:2/6 MOD-->CHIDX#  1
#2/7:6.28s; CLI:2/7 MOD-->CHIDX#  1
#2/8:6.40s; CLI:2/8 MOD-->CHIDX#  1
#3/0:6.42s; MIN:3/0 MOD-->CHIDX#  1
#3/1:6.35s; CLI:3/1 MOD-->CHIDX#  1
#3/2:6.58s; CLI:3/2 MOD-->CHIDX#  1
#3/3:6.37s; CLI:3/3 MOD-->CHIDX#  1
#3/4:6.61s; MIN:3/4 MOD-->CHIDX#  1
#3/5:7.05s; CLI:3/5 MOD-->CHIDX#  1
#3/6:6.34s; CLI:3/6 MOD-->CHIDX#  1
#3/7:6.55s; CLI:3/7 MOD-->CHIDX#  1
#3/8:6.17s; CLI:3/8 MOD-->CHIDX#  1
#4/0:6.59s; CLI:4/0 MOD-->CHIDX#  1
#4/1:6.33s; CLI:4/1 MOD-->CHIDX#  1
#4/2:6.59s; CLI:4/2 MOD-->CHIDX#  1
#4/3:6.27s; CLI:4/3 MOD-->CHIDX#  1
#4/4:6.95s; CLI:4/4 MOD-->CHIDX#  1
#4/5:6.59s; CLI:4/5 MOD-->CHIDX#  1
#4/6:6.39s; CLI:4/6 MOD-->CHIDX#  1
#4/7:6.44s; CLI:4/7 MOD-->CHIDX#  1
#

In [7]:
def predict():
    txs = []
    for t in blockchain.lastBlock.transactions:
        if t.type == 'MOD':
            txs.append(t.load)
    for c in blockchain.cli_pointers:
        if c.alias[-1] in ['3','4']:
            model = CNNLSTM(36,2)
            weights = c.aggregate(txs)
            dev = c.predict(model, weights)
            print(c.alias, dev)

predict()

A:2/3 2/3 [0.5165]
A:2/4 2/4 [0.1216]
A:3/3 3/3 [0.1327]
A:3/4 3/4 [0.0908]
A:4/3 4/3 [0.9043]
A:4/4 4/4 [0.1863]
A:5/3 5/3 [0.1663]
A:5/4 5/4 [0.0996]
